In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial.distance import hamming
import itertools
from helper.bitstring_helper import factoradic_to_permutation, segmented_sizes_for_factoradic, bitstring_to_int_list, \
    repair_factoradic_with_modulo, repair_factoradic_with_scaling, bits_needed_for_integer
from helper.bitstring_helper import bitstring_to_single_int, single_int_to_factoradic, \
    bits_needed_for_permutation_as_int
from collections import Counter, defaultdict
import math
from Levenshtein import distance as levenshtein_distance


dBlue = "#003f6b"
lBlue = "#00abda"
green = "#95c11f"
dGreen = "#00786b"
orange = "#d53d0e"
yellow = "#fdc300"

colors = [dBlue, lBlue, green, dGreen]
plt.rcParams['figure.dpi'] = 200


def encoding_a(bitstring, segment_sizes, use_gray):
    route = bitstring_to_int_list(bitstring, segment_sizes, use_gray)
    route = repair_factoradic_with_modulo(route)
    return factoradic_to_permutation(route)


def encoding_b(bitstring, segment_sizes, max_segment_sizes, use_gray):
    route = bitstring_to_int_list(bitstring, segment_sizes, use_gray)
    route = repair_factoradic_with_scaling(route, segment_sizes, max_segment_sizes)
    return factoradic_to_permutation(route)


def encoding_c(bitstring, n, use_gray):
    route = bitstring_to_single_int(bitstring, use_gray)
    route = single_int_to_factoradic(route, n)
    return factoradic_to_permutation(route, False)


def kendall_distance(perm1, perm2):
    n = len(perm1)
    index_map = {value: idx for idx, value in enumerate(perm2)}
    mapped_perm1 = np.array([index_map[value] for value in perm1])
    inversions = 0
    for i in range(n):
        for j in range(i + 1, n):
            if mapped_perm1[i] > mapped_perm1[j]:
                inversions += 1
    return inversions


def break_distance(perm1, perm2):
    def get_adjacent_pairs(permutation):
        return [(permutation[i], permutation[i + 1]) for i in range(len(permutation) - 1)]

    # Get adjacent pairs for both permutations
    pairs1 = get_adjacent_pairs(perm1)
    pairs2 = get_adjacent_pairs(perm2)

    # Reverse each pair in perm2
    pairs2_reversed = [(b, a) for (a, b) in pairs2]

    # Use set operations for faster lookups
    set_pairs2 = set(pairs2)
    set_pairs2_reversed = set(pairs2_reversed)

    # Count common pairs
    common_pairs = sum(1 for pair in pairs1 if pair in set_pairs2 or pair in set_pairs2_reversed)

    # Calculate break distance
    total_pairs = len(pairs1)
    break_distance = total_pairs - common_pairs

    return break_distance


In [ ]:
problem_sizes = np.arange(2, 201)

def plot_bit_scaling(problem_sizes, method):
    ab_x = [np.sum(segmented_sizes_for_factoradic(n-1)) for n in problem_sizes]
    c_x = [bits_needed_for_permutation_as_int(n)[0] for n in problem_sizes]

    plt.figure(figsize=(8, 6))
    plt.plot(problem_sizes, ab_x, label='Encoding A', color=colors[0])
    plt.plot(problem_sizes, ab_x, label='Encoding B', color=colors[1], linestyle='dotted')
    plt.plot(problem_sizes, c_x, label='Encoding C', color=colors[3])
    plt.xlabel('Problem Size', fontsize=16)
    plt.ylabel('Number of Bits Needed', fontsize=16)
    if method in [0, 1]:
        plt.yscale('log')
    if method == 1:
        plt.xscale('log')
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.show()


for i in range(0, 3):
    plot_bit_scaling(problem_sizes, i)

In [ ]:
def compression_ratio_formula_ab(n, num_permutations):
    """Calculate the compression ratio based on the number of bits needed and the number of permutations for encodings A and B."""
    b = int(np.sum(segmented_sizes_for_factoradic(n-1)))
    num_bitstring_states = 2 ** b
    return num_bitstring_states / num_permutations


def compression_ratio_formula_c(n, num_permutations):
    """Calculate the compression ratio based on the number of bits needed and the number of permutations for encoding C."""
    b = int(bits_needed_for_permutation_as_int(n)[0])
    num_bitstring_states = 2 ** b
    return num_bitstring_states / num_permutations


def plot_compression_ratio_ab(n_values, compression_ratio_ab):
    # Plotting the results
    plt.figure(figsize=(10, 7))
    plt.plot(n_values, compression_ratio_ab, label="Encoding A",color=colors[0])
    plt.plot(n_values, compression_ratio_ab, label="Encoding B",color=colors[1], linestyle='dotted')
    plt.xlabel('Problem Size', fontsize=16)
    plt.ylabel('Redundancy Ratio', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.yscale('log')
    plt.xscale('log')
    plt.grid(True)
    plt.xlim(0, max(n_values))
    plt.ylim(1, max(compression_ratio_ab))
    plt.legend()
    plt.show()

def plot_compression_ratio_c(n_values, compression_ratios_c):
    # Plotting the results
    plt.figure(figsize=(10, 7), dpi=200)
    plt.plot(n_values, compression_ratios_c, label="Encoding C", color=colors[3])
    plt.xlabel('Problem Size', fontsize=16)
    plt.ylabel('Redundancy Ratio', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.axhline(y=1, color=orange, linestyle='--', linewidth=1)
    plt.axhline(y=2, color=orange, linestyle='--', linewidth=1)
    plt.grid(True)

    plt.xlim(0, max(n_values))
    plt.ylim(0.85, 2.15)
    plt.legend()
    plt.show()


n_values = range(2, 201)
compression_ratios_ab = []
compression_ratios_c = []
compression_ratios_d = []

for n in n_values:
    num_permutations = int(math.factorial(n))
    compression_ratios_ab.append(compression_ratio_formula_ab(n, num_permutations))
    compression_ratios_c.append(compression_ratio_formula_c(n, num_permutations))
    # print(n, compression_ratios_ab[-1], compression_ratios_c[-1], compression_ratios_d[-1])

plot_compression_ratio_ab(n_values, compression_ratios_ab)
plot_compression_ratio_c(n_values, compression_ratios_c)

In [ ]:
n = 5
factor_segment_sizes = segmented_sizes_for_factoradic(n-1)
bitstrings_ab = [np.array(bits) for bits in itertools.product([0, 1], repeat=np.sum(factor_segment_sizes))]
max_segment_sizes = np.arange(n - 1, 0, -1)
bitstrings_c = [np.array(bits) for bits in itertools.product([0, 1], repeat=bits_needed_for_permutation_as_int(n)[0])]
d1_segmented_sizes = [bits_needed_for_integer(1)]*n
d3_segmented_sizes = [bits_needed_for_integer(3)]*n

In [ ]:
def mutate_bitstring(bitstring, index):
    """Flip a single bit in the bitstring at the specified index, then revert it after mutation."""
    bitstring[index] = 1 - bitstring[index]  # Flip the bit
    return bitstring


def mutation_impact_analysis(encoding_fn, bitstrings, distance_meas, *encoding_args):
    """Analyze the impact of a single-bit mutation on the resulting permutation with optimizations."""
    impacts = np.zeros((len(bitstrings), len(bitstrings[0])))  # Initialize a 2D array for impacts

    for bit_idx, bitstring in enumerate(bitstrings):
        original_permutation = encoding_fn(bitstring, *encoding_args)
        for i in range(len(bitstring)):
            mutated_bitstring = mutate_bitstring(bitstring.copy(), i)
            mutated_permutation = encoding_fn(mutated_bitstring, *encoding_args)
            result = distance_meas(original_permutation, mutated_permutation)
            impacts[bit_idx, i] = result if not isinstance(result, tuple) else result[0]
            # print(result, original_permutation, mutated_permutation)
    return impacts


def plot_mutation(hist_data_dict, dist_name):
    all_data = np.concatenate([data.flatten() for data in hist_data_dict.values()])
    min_val, max_val = all_data.min(), all_data.max()
    bins = np.arange(min_val, max_val + 2) - 0.5

    # Creating grouped bar chart data
    bin_labels = np.arange(min_val, max_val + 1)
    width = 0.3  # Width of each bar
    offsets = np.arange(-(len(hist_data_dict) - 1) / 2, (len(hist_data_dict) + 1) / 2) * width

    for i, (label, data) in enumerate(hist_data_dict.items()):
        counts, _ = np.histogram(data.flatten(), bins=bins)
        plt.bar(bin_labels + offsets[i], counts, width=width, alpha=0.8, label=label, color=colors[i])

    plt.xlabel(dist_name, fontsize=16)
    plt.ylabel('Count', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend()
    plt.xticks(bin_labels)
    plt.show()


dists = {
    'Hamming Distance': lambda x, y: hamming(x, y) * len(x),
    'Kendall \u03C4 Distance': kendall_distance,
    'Break Distance': break_distance,
    'Levenshtein Distance': levenshtein_distance,

}


def save_mutation_data(hist_data_dict):
    mutation_count_data = defaultdict(lambda: defaultdict(int))  # Nested defaultdict for storing counts

    for encoding_type, data in hist_data_dict.items():
        flattened_data = data.flatten()
        unique, counts = np.unique(flattened_data, return_counts=True)
        for dist_val, count in zip(unique, counts):
            mutation_count_data[encoding_type][dist_val] += count

    return mutation_count_data


mutation_results = defaultdict(dict)
mutation_results_gray = defaultdict(dict)
for dist_name, distance_meas in dists.items():
    impacts_a = mutation_impact_analysis(encoding_a, bitstrings_ab, distance_meas, factor_segment_sizes, False)
    impacts_b = mutation_impact_analysis(encoding_b, bitstrings_ab, distance_meas, factor_segment_sizes, max_segment_sizes,
                                         False)
    impacts_c = mutation_impact_analysis(encoding_c, bitstrings_c, distance_meas, n, False)

    impacts_ag = mutation_impact_analysis(encoding_a, bitstrings_ab, distance_meas, factor_segment_sizes, True)
    impacts_bg = mutation_impact_analysis(encoding_b, bitstrings_ab, distance_meas, factor_segment_sizes, max_segment_sizes,
                                          True)
    impacts_cg = mutation_impact_analysis(encoding_c, bitstrings_c, distance_meas, n, True)

    mutation_data = {'Encoding A': impacts_a, 'Encoding B': impacts_b, 'Encoding C': impacts_c}
    mutation_data_gray = {'Encoding AG': impacts_ag, 'Encoding BG': impacts_bg, 'Encoding CG': impacts_cg}

    plot_mutation(mutation_data, dist_name)
    plot_mutation(mutation_data_gray, dist_name)
    mutation_results[dist_name] = save_mutation_data(mutation_data)
    mutation_results_gray[dist_name] = save_mutation_data(mutation_data_gray)


In [ ]:
n = 5
segment_sizes = segmented_sizes_for_factoradic(n-1)
bitstrings_ab = [np.array(bits, dtype=object) for bits in itertools.product([0, 1], repeat=sum(segment_sizes)) ]

max_segment_sizes = np.arange(n - 1, 0, -1)
bitstrings_c = [np.array(bits, dtype=object) for bits in itertools.product([0, 1], repeat=bits_needed_for_permutation_as_int(n)[0])]

def plot_permutation_histograms(n, bitstrings_ab, bitstrings_c, encoding_a, encoding_b, encoding_c, colors, segment_sizes, max_segment_sizes):
    # Collect the permutation counts for each encoding
    all_permutations_a = [tuple(encoding_a(bitstring, segment_sizes, False)) for bitstring in bitstrings_ab]
    all_permutations_b = [tuple(encoding_b(bitstring, segment_sizes, max_segment_sizes, False)) for bitstring in bitstrings_ab]
    all_permutations_c = [tuple(encoding_c(bitstring, n, False)) for bitstring in bitstrings_c]
    #
    # Count the occurrences of each permutation for each encoding
    permutation_counts_a = Counter(all_permutations_a)
    permutation_counts_b = Counter(all_permutations_b)
    permutation_counts_c = Counter(all_permutations_c)
    print(max(permutation_counts_a.values()))

    # Sort the permutations to maintain the same order across all plots
    all_permutations = sorted(set(permutation_counts_a.keys()).union(permutation_counts_b.keys(), permutation_counts_c.keys()))

    # Align counts with the sorted permutations
    counts_a = [permutation_counts_a.get(p, 0) for p in all_permutations]
    counts_b = [permutation_counts_b.get(p, 0) for p in all_permutations]
    counts_c = [permutation_counts_c.get(p, 0) for p in all_permutations]

    # Plot the histograms in subplots
    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True, gridspec_kw={'hspace': 0})  # Increase the figure size
    labels = [str(p) for p in all_permutations]


    # Plot each histogram
    axes[0].bar(range(len(all_permutations)), counts_a, color=colors[0], label='Encoding A')
    axes[1].bar(range(len(all_permutations)), counts_b, color=colors[1], label='Encoding B')
    axes[2].bar(range(len(all_permutations)), counts_c, color=colors[3], label='Encoding C')

    # Set labels, titles, and y-axis ticks for each subplot
    for i, ax in enumerate(axes):
        ax.set_ylabel('Counts', labelpad=10, fontsize=13)  # Add padding to the y-axis label
        max_count = max([counts_a, counts_b, counts_c][i])
        ax.set_yticks(range(0, max_count + 1))  # Set y-ticks as integers
        ax.legend(loc='upper right')
        ax.set_xlim(left=-1, right=len(all_permutations))

    # Configure x-axis for the bottom plot
    axes[2].set_xticks(range(len(all_permutations)))
    axes[2].set_xticklabels(labels, rotation=90, ha='right', fontsize=9)
    axes[2].set_xlabel('Permutations', fontsize=13)
    # Adjust layout to prevent overlap
    plt.tight_layout(pad=2.0)  # Add extra padding to prevent y-axis tick overlap
    plt.subplots_adjust(bottom=0.25)  # Add space at the bottom for x-axis labels
    plt.show()

# Example usage with the provided parameters
plot_permutation_histograms(n, bitstrings_ab, bitstrings_c, encoding_a, encoding_b, encoding_c, colors, segment_sizes, max_segment_sizes)

In [ ]:
def plot_max_redundancy(n_values, encoding_a, colors):
    max_counts = []

    for n in n_values:
        segment_sizes = segmented_sizes_for_factoradic(n - 1)
        bitstrings_ab = [np.array(bits, dtype=object) for bits in itertools.product([0, 1], repeat=sum(segment_sizes))]
        all_permutations_a = [tuple(encoding_a(bitstring, segment_sizes, False)) for bitstring in bitstrings_ab]
        permutation_counts_a = Counter(all_permutations_a)

        # Get the maximum count of any permutation
        max_count = max(permutation_counts_a.values())
        max_counts.append(max_count)
        print(n, np.sum(segment_sizes), max_count)

    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.plot(n_values, max_counts, marker='o', color=colors[0], label='Maximum Redundancy')
    plt.xlabel('Problem Size')
    plt.ylabel('Maximum Redundancy of a Single Permutation')
    plt.xticks(n_values)
    plt.legend()
    plt.grid(True)
    plt.show()


n_values = range(2, 10)
plot_max_redundancy(n_values, encoding_a, colors)



In [ ]:
def mutation_impact_analysis_on_int(bitstrings, gray):
    """Analyze the impact of a single-bit mutation on the resulting permutation with optimizations."""
    impacts = np.zeros((len(bitstrings), len(bitstrings[0])))  # Initialize a 2D array for impacts

    for bit_idx, bitstring in enumerate(bitstrings):
        original = bitstring_to_single_int(bitstring, gray)
        for i in range(len(bitstring)):
            mutated_bitstring = mutate_bitstring(bitstring.copy(), i)
            mutated = bitstring_to_single_int(mutated_bitstring, gray)
            # print(gray, bitstring, mutated_bitstring, original, mutated, abs(original - mutated))
            result = abs(original - mutated)
            impacts[bit_idx, i] = result if not isinstance(result, tuple) else result[0]
    return impacts


data = {"Encoding C": mutation_impact_analysis_on_int(bitstrings_c, False),
        "Encoding CG": mutation_impact_analysis_on_int(bitstrings_c, True)}


def plot(data):
    plt.figure(figsize=(8, 6))  # Adjust the figure size as needed
    for (label, values), color in zip(data.items(), colors[-2:]):
        counts, bin_edges = np.histogram(values.flatten(), bins=128)
        plt.bar(bin_edges[:-1], counts, width=1, alpha=0.8, color=color, label=label)  #np.diff(bin_edges)
    plt.xlabel('Distance', fontsize=16)
    plt.ylabel('Counts', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlim(left=0)
    plt.legend(fontsize=12)
    plt.grid()
    plt.show()

def plot_cum(data):
    # Plotting the cumulative distribution of mutation impacts
    fig, ax = plt.subplots(figsize=(8, 6))

    for idx, (encoding, impacts) in enumerate(data.items()):
        flat_impacts = impacts.flatten()
        sorted_impacts = np.sort(flat_impacts)
        cumulative_percent = np.cumsum(sorted_impacts) / np.sum(sorted_impacts) * 100
        # Use the corresponding color from the list, defaulting if the list is too short
        ax.plot(sorted_impacts, cumulative_percent, label=encoding, color=colors[-2:][idx])

    ax.set_xlabel('Distance', fontsize=14)
    ax.set_ylabel('Cumulative Percentage (%)',  fontsize=14)
    ax.legend(fontsize=12)
    plt.xlim(left=0)
    plt.grid()
    plt.show()


plot(data)
plot_cum(data)

In [ ]:
def print_cumulative_counts(data):
    """Calculate and print the cumulative sum of counts at specified distances."""
    distances = [1, 2, 4, 8, 16, 32, 64]  # The specific distances to consider

    for encoding, impacts in data.items():
        print(f"\nEncoding: {encoding}")
        flat_impacts = impacts.flatten()

        # Calculate the histogram
        counts, bin_edges = np.histogram(flat_impacts, bins=128)

        # Calculate the cumulative sum of counts
        cumulative_counts = np.cumsum(counts)
        total_counts = cumulative_counts[-1]  # The total number of counts

        # Find cumulative counts at the specified distances
        for distance in distances:
            # Find the bin index closest to the current distance
            bin_index = np.searchsorted(bin_edges, distance, side='right') - 1
            if bin_index < 0 or bin_index >= len(cumulative_counts):
                print(f"Distance {distance}: Out of bounds")
                continue

            cumulative_percentage = cumulative_counts[bin_index] / total_counts * 100
            print(f"Distance {distance}: Cumulative Percentage = {cumulative_percentage:.2f}%")

print_cumulative_counts(data)

In [ ]:
import numpy as np

class VehicleRoutingProblem:
    def __init__(self, filename: str):
        self.filename = filename
        self.data = None
        self.distance_matrix = None
        self.length = 0
        self.load_data()

    def load_data(self):
        # Load data from the file
        # The 'data' attribute is assumed to be structured as [[x, y, demand], ...] including the depot
        # Here we're simulating loading data (in practice you'd load from a file)
        self.data = np.array([[0, 0, 0],  # Depot
                              [10, 10, 5],  # Customer 1
                              [20, 20, 10],  # Customer 2
                              [30, 30, 15]])  # Customer 3

        # Compute distance matrix based on coordinates (x, y)
        self.distance_matrix = self.compute_distance_matrix(self.data)
        self.length = len(self.data)  # Number of locations (including depot)

    def compute_distance_matrix(self, data):
        """
        Computes a distance matrix between all points (including the depot).
        """
        n = len(data)
        dist_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                dist_matrix[i][j] = np.sqrt((data[i][0] - data[j][0]) ** 2 + (data[i][1] - data[j][1]) ** 2)
        return dist_matrix

def split_route(vrp: VehicleRoutingProblem, global_route, vehicle_capacity):
    """
    Splits a given global route into feasible routes based on vehicle capacity.

    Args:
        vrp: The VehicleRoutingProblem instance with data and distance matrix.
        global_route: List of customer indices representing the global route (without depot).
        vehicle_capacity: The capacity of the vehicle.

    Returns:
        predecessor_list: List of predecessors for route reconstruction.
        min_cost: List of minimum costs to reach each customer.
    """
    num_customers = len(global_route)  # Number of customers in the route
    min_cost = [float('inf')] * (num_customers + 1)  # Minimum cost to reach each customer
    predecessor_list = [None] * (num_customers + 1)  # List of predecessors for optimal route reconstruction
    min_cost[0] = 0  # Start at the depot, cost to reach depot is 0

    # Loop through each customer `i` in the sequence
    for i in range(1, num_customers + 1):
        total_demand = 0
        j = i
        while j <= num_customers and total_demand + vrp.data[global_route[j-1], 2] <= vehicle_capacity:
            total_demand += vrp.data[global_route[j-1], 2]
            # Calculate the cost of the route segment from customer i to j
            if j == i:
                # If it's a single customer, cost is depot -> customer -> depot
                trip_cost = (vrp.distance_matrix[0][global_route[i-1]]  # Depot -> customer i
                             + vrp.distance_matrix[global_route[i-1]][0])  # Customer i -> depot
            else:
                # For multiple customers, calculate the trip cost
                trip_cost = (vrp.distance_matrix[0][global_route[i-1]]  # Depot -> first customer in the segment
                             + sum(vrp.distance_matrix[global_route[k-1]][global_route[k]] for k in range(i, j-1))  # Customer chain
                             + vrp.distance_matrix[global_route[j-1]][0])  # Last customer -> depot

            # Update minimum cost and predecessor if a better path is found
            if min_cost[i-1] + trip_cost < min_cost[j]:
                min_cost[j] = min_cost[i-1] + trip_cost
                predecessor_list[j] = i-1

            j += 1

    # Return predecessor list and minimum costs
    return predecessor_list, min_cost

def extract_routes(global_route, predecessor_list):
    """
    Extracts the optimal routes from the predecessor list.

    Args:
        global_route: List of customer indices representing the global route (without depot).
        predecessor_list: Predecessor list returned by the split method.

    Returns:
        routes: A list of routes, where each route is a list of customer indices starting and ending with the depot.
    """
    routes = []
    current_index = len(global_route)  # Start from the last customer
    while current_index > 0:
        previous_index = predecessor_list[current_index]
        # Extract the customers between previous_index and current_index
        route_segment = global_route[previous_index:current_index]
        # Add the depot at the start and end of the route
        route = [0] + route_segment + [0]
        routes.append(route)
        # Move to the predecessor
        current_index = previous_index

    # Reverse routes to get them in correct order
    return routes[::-1]  # Since routes are constructed backwards

# Example usage
if __name__ == '__main__':
    # Create the VRP instance
    vrp = VehicleRoutingProblem('dummy_file.txt')

    # Customer sequence (global route without depot)
    global_route = [1, 2, 3]

    # Perform split based on vehicle capacity
    vehicle_capacity = 20
    predecessor_list, min_cost = split_route(vrp, global_route, vehicle_capacity)


    # Extract routes from predecessors
    routes = extract_routes(global_route, predecessor_list)

    # Print the routes and total cost
    print("Optimal Routes:", routes)
    print("Minimum Cost to Reach Each Customer:", min_cost)
    print("Total Cost of the Solution:", min_cost[-1])  # Total cost is the cost to reach the last customer
